In [1]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
import math
# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [3]:
df = pd.read_csv('./NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013 - 13tbl8ny.csv', 
                 thousands=',', skiprows=3, header = 1)
df.columns  = ['City',
 'Population',
 'Violent_crime',
 'Murder',
 'Rape_revised_definition',
 'Rape',
 'Robbery',
 'Aggravated_assault',
 'Property_crime',
 'Burglary',
 'Larceny_theft',
 'Motor_vehicle_theft',
 'Arson3']
# drop last 3 rows
df = df.iloc[:348,:]
df.drop("Rape_revised_definition", 1, inplace=True)
df.head()

,City,Population,Violent_crime,Murder,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny_theft,Motor_vehicle_theft,Arson3
0,Adams Village,1861.000,0.000,0.000,0.000,0.000,0.000,12.000,2.000,10.000,0.000,0.000
1,Addison Town and Village,2577.000,3.000,0.000,0.000,0.000,3.000,24.000,3.000,20.000,1.000,0.000
2,Akron Village,2846.000,3.000,0.000,0.000,0.000,3.000,16.000,1.000,15.000,0.000,0.000
3,Albany,97956.000,791.000,8.000,30.000,227.000,526.000,4090.000,705.000,3243.000,142.000,nan
4,Albion Village,6388.000,23.000,0.000,3.000,4.000,16.000,223.000,53.000,165.000,5.000,nan


In [35]:
df.describe()

,Population,Violent_crime,Murder,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny_theft,Motor_vehicle_theft,Arson3,y_hat,resid,pred_off_percent,has_violent_crime,has_murder_nonnegligent_manslaughter
count,348.000,348.000,348.000,348.000,348.000,348.000,348.000,348.000,348.000,348.000,187.000,348.000,348.000,342.000,348.000,348.000
mean,40037.632,201.595,1.566,5.865,72.902,121.261,792.606,119.684,637.017,35.905,1.872,792.606,-0.000,3.398,0.853,0.141
std,450037.368,2815.269,18.304,60.425,1031.033,1706.132,7659.725,924.949,6346.054,403.424,10.693,7653.764,302.135,12.527,0.354,0.348
min,526.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,95.698,-1509.953,-0.714,0.000,0.000
25%,3003.000,2.000,0.000,0.000,0.000,1.000,40.500,6.000,31.000,0.000,0.000,126.831,-104.154,-0.093,1.000,0.000
50%,7233.500,6.000,0.000,0.000,1.000,4.000,112.500,17.500,94.000,2.000,0.000,173.853,-75.341,0.728,1.000,0.000
75%,18427.500,22.000,0.000,2.000,5.000,14.000,341.000,51.250,287.250,7.000,1.000,316.780,29.460,2.342,1.000,0.000
max,8396126.000,52384.000,335.000,1112.000,19170.000,31767.000,141971.000,16606.000,117931.000,7434.000,132.000,142089.524,1770.475,151.876,1.000,1.000


In [5]:
formula1 = "Property_crime ~ Population + Murder + Robbery + Violent_crime"
# Fit the model to our data using the formula.
lm1 = smf.ols(formula=formula1, data=df).fit()
print("Model Parameters\n===========================================")
print(lm1.params)
print("p-values\n===========================================")
print(lm1.pvalues)
print("\n\nR-Squared: {}".format(lm1.rsquared))

Model Parameters
Intercept        90.151
Population        0.011
Murder          241.823
Robbery          -8.668
Violent_crime     2.646
dtype: float64
p-values
Intercept       0.000
Population      0.000
Murder          0.000
Robbery         0.000
Violent_crime   0.000
dtype: float64


R-Squared: 0.998444118611959


I highly doubt all the p-values are 0.0, so What are the actual p-values? 

In [23]:
# let's take a deeper look into where the model fails the most
df["y_hat"] = lm1.predict(df)
df["resid"] = lm1.resid
df["pred_off_percent"] = (df.y_hat - df.Property_crime)/df.Property_crime
# need to replace 0's with nan
indices = list(np.where((df.pred_off_percent > 10000))[0])
df.iloc[indices, 14] = np.nan

In [32]:
print("Predicted Property Crime is off by {}%".format(round(df.pred_off_percent.mean()*100,1)))
print("...of off by an average of {} crimes".format(round(np.abs(df.resid).mean(),1)))

Predicted Property Crime is off by 339.8%
...of off by an average of 173.9 crimes


In [39]:
# looks like one pitfall of this model is handling situations where
# property crime is zero
df[df.Property_crime == 0].mean()
# so when property_cime is 0, so are all else crime metrics...

Population                         1251.333
Violent_crime                         0.000
Murder_nonnegligent_manslaughter      0.000
Rape_legacy_definition                0.000
Robbery                               0.000
Aggravated_assault                    0.000
Property_crime                        0.000
Burglary                              0.000
Larceny_theft                         0.000
Motor_vehicle_theft                   0.000
Arson3                                0.000
y_hat                               103.347
resid                              -103.347
pred_off_percent                        inf
dtype: float64

So when a city has zero crimes (odd) this model is off by an average of 103.3 crimes...not good (this group only consists of 6 cities but still needs to be accounted for).

In [38]:
# Make a no_crime dummy var for each crime 
df["has_violent_crime"] = np.where(df.Violent_crime>0,1,0)
df["has_murder"] = np.where(df.Murder>0,1,0)
df["has_rape"] = np.where(df.Rape>0,1,0)
df["has_robbery"] = np.where(df.Robbery>0,1,0)
df["has_aggravated_assault"] = np.where(df.Aggravated_assault>0,1,0)
df["has_burglary"] = np.where(df.Burglary>0,1,0)

Now update the previous formula to account for the situation where there is no crime in that category.

In [39]:
# now just 
formula2 = "Property_crime ~ Population + has_murder*Murder + has_robbery*Robbery + has_violent_crime*Violent_crime"

lm2 = smf.ols(formula=formula2, data=df).fit()
print("Model Parameters\n===========================================")
print(lm1.params)
print("p-values\n===========================================")
print(lm1.pvalues)
print("\n\nR-Squared: {}".format(lm1.rsquared))

Model Parameters
Intercept        90.151
Population        0.011
Murder          241.823
Robbery          -8.668
Violent_crime     2.646
dtype: float64
p-values
Intercept       0.000
Population      0.000
Murder          0.000
Robbery         0.000
Violent_crime   0.000
dtype: float64


R-Squared: 0.998444118611959


In [74]:
df2 = df.copy()
df2["y_hat"] = lm2.predict(df2)
df2["resid"] = lm2.resid
df2["pred_off_percent"] = (df2.y_hat - df2.Property_crime)/df2.Property_crime
indices = list(np.where((df2.pred_off_percent == -(math.inf)) | (df2.pred_off_percent == math.inf))[0])
df2.iloc[indices,14] = np.nan

In [78]:
print("Predicted Property Crime is off by {}%".format(round(df2.pred_off_percent.mean()*100,1)))
print("Or off by an average of {} crimes".format(round(np.abs(df2.resid).mean(),1)))

Predicted Property Crime is off by 89.3%
Or off by an average of 153.4 crimes


Notice that this is a significant difference in the average percentage the model is off (334 down to 89), however the number of crimes is still relatively high (173 down to 153).

In [82]:
# include every variable except 5,6, and 8
formula_saturated = "Property_crime ~ " + ' + '.join(df.columns[1:4]) + " + " + df.columns[7] + " + " + ' + '.join(df.columns[9:])
lm3 = smf.ols(formula=formula_saturated, data=df).fit()
print("Model Parameters\n===========================================")
print(lm3.params)
print("p-values\n===========================================")
print(lm3.pvalues)
print("\n\nR-Squared: {}".format(lm3.rsquared))

Model Parameters
Intercept                              -0.000
Population                              0.000
Violent_crime                           0.000
Murder                                 -0.000
Property_crime                          0.667
Larceny_theft                          -0.000
Motor_vehicle_theft                    -0.000
Arson3                                 -0.000
y_hat                                   0.333
resid                                   0.333
pred_off_percent                        0.000
has_violent_crime                       0.000
has_murder_nonnegligent_manslaughter   -0.000
has_rape_legacy_definition             -0.000
has_robbery                            -0.000
has_aggravated_assault                 -0.000
has_burglary                           -0.000
has_murder                              0.000
has_rape                               -0.000
dtype: float64
p-values
Intercept                              0.977
Population                             0